In [1]:
import pandas as pd

dataset = pd.read_csv("C:/Users/elygl/Documents/mlclass-master/mlclass-master/01_Preprocessing/diabetes_dataset.csv")

In [2]:
dataset.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148.0,72.0,35.0,NaN,33.6,0.627,50,1
1,1,85.0,66.0,29.0,NaN,26.6,0.351,31,0
2,8,183.0,64.0,NaN,NaN,23.3,0.672,32,1
3,0,137.0,40.0,35.0,168.0,43.1,2.288,33,1
4,5,116.0,74.0,NaN,NaN,25.6,0.201,30,0


## View the amount of missing data

In [23]:
dataset.isnull().sum()

Pregnancies                   0
Glucose                       5
BloodPressure                35
SkinThickness               227
Insulin                     374
BMI                          11
DiabetesPedigreeFunction      0
Age                           0
Outcome                       0
dtype: int64

## Percentage of missing data


In [31]:
size = dataset.shape[0]*8
missing_data = 0
col = ['Pregnancies',"Glucose",'BloodPressure','SkinThickness',
       'Insulin','BMI','DiabetesPedigreeFunction','Age']
for i in col:
    missing_data += dataset[i].isnull().sum()
"The amount of missing data in the dataset represents {:%} of 4.576 data".format(missing_data/size)

'The amount of missing data in the dataset represents 14.248252% of 4.576 data'